# Human coding

In this notebook, we will work with the following aspects of human coding:

1. Revisiting our identifier example 
1. Human coding design
1. Heuristics
1. Cartesian joins

In [1]:
import pandas as pd

# Identifier example

Returning to our identifier example from the planning segment, we can look through some elements of the design of a coding spreadsheet.

Note: I removed one year from each firm for brevity.

In [2]:
coding = pd.DataFrame([
    {'name': 'Apple',
     'year': '2018'},
    {'name': 'Apple',
     'year': '2019'},
    {'name': 'Microsoft',
     'year': '2018'},
    {'name': 'Microsoft',
     'year': '2019'},
    {'name': 'Berkshire Hathaway',
     'year': '2018'},
    {'name': 'Berkshire Hathaway',
     'year': '2019'}
])

coding.head(10)

,name,year
0,Apple,2018
1,Apple,2019
2,Microsoft,2018
3,Microsoft,2019
4,Berkshire Hathaway,2018
5,Berkshire Hathaway,2019


In [3]:
# We can get unique values of a column with the unique method.
coding['name'].unique()

array(['Apple', 'Microsoft', 'Berkshire Hathaway'], dtype=object)

In [4]:
# We can make a new dataframe with those unique values.
code_table = pd.DataFrame(coding['name'].unique())
code_table = code_table.rename(columns={0: 'name'})
code_table.head()

,name
0,Apple
1,Microsoft
2,Berkshire Hathaway


In [5]:
# Then, we can populate new columns for use in coding.
for new_col in ['gvkey', 'source', 'coder', 'flag', 'notes']:
    code_table[new_col] = ''
code_table.head()

,name,gvkey,source,coder,flag,notes
0,Apple,,,,,
1,Microsoft,,,,,
2,Berkshire Hathaway,,,,,


# Human coding design

When we design coding processes for humans, there are a few important things that we need to manage and balance.

1. Cost/time spent
1. Future use and versatility
1. Consistency across time and coders
1. Cleanness of data
1. Identification of errors


There are a few practices that I suggest, which we can see embedded in the example above.

1. **Minimize what you present to coders.** This table has a lot less than a typical dataset, because we only want to provide what is needed to do the work. While we don't have to be extremely strict about this (i.e. including a column for meaningful context but not direct use is sometimes helpful), everything you add has a cost in time.
1. **Communicate a benchmark for completing work.** Coders are sometimes less than particularly diligent (and it's fair to say that this is often tedious work). I have found that communicating a specific benchmark, even a generous one (I tend to multiply my average over a couple dozen observations by 2.0), improves output. You can use things like file creation times or Dropbox revisions (depending on the design) to compute the time as a way of measuring compliance.
1. **Capture information that helps improve the process.** It may not be strictly necessary to capture the source here, but it may be useful to have. For example, if we had a cascading design, we might find that some sources are not helpful enough to include going forward.
1. **Provide explicit instructions.** Instructions help the same person perform similarly over time, and they help us keep multiple coders in sync. They can also be helpful to show to reviewers to give face validity to the rigor of the management of the coding protocol.
1. **Give coders a mechanism to raise issues and capture the unexpected.** Here, I use a flag column and a notes column. The idea is that a flag tells me to look at it, and notes capture something unexpected. I advise coders that using this is rare, but it's there to surface things that they notice. Without this, you sometimes get responses like (for example, for gvkey) "001234 or 056789." This will read in as the wrong type, and it won't merge properly (even if one of them is correct). By giving them a mechanism (and instructions) to communicate, they can choose the better of the two, flag it, and make a note.
1. **Establish a review process to detect errors.** I typically have two forms of review. First, when onboarding a new coder, we have them do a short list of coding tasks (from the real data) that we use for every new coder. We use this to detect errors in process or conscientiousness early and correct them, and it's common to have issues (around 50 percent of new coders have at least one issue, in my experience). Using the same list means that we know those particular issues. Second, for more nuanced/complex projects, we write instructions for reviewing the work that is submitted.

# Heuristics

Sometimes, we have an imperfect way of coding a variable that works a reasonable amount of the time.
The idea here is that we can pre-populate those heuristic values, and we ask coders to check them, leave them if they are correct, and correct them if not.

It's worth noting that this necessarily increases the nuance and complexity of a coding task.
On the other hand, if the coders are not unduly anchored by the heuristic, it can create a substantial time savings by reducing data entry.
These issues make me more likely to use this technique with coders who we know to do good work, PhD students, and other co-authors.

In [6]:
msft_nyt = pd.read_csv('../data/msft_nyt.csv')
msft_nyt.head()

,_id,byline.organization,byline.original,byline.person,document_type,headline.content_kicker,headline.kicker,headline.main,headline.name,headline.print_headline,...,pub_date,score,section_name,snippet,source,type_of_material,uri,web_url,word_count,id_ticker
0,5bbb75cb068401528a2df0f5,REUTERS,By REUTERS,[],article,NaN,NaN,Microsoft Testing New Game-Streaming Service,NaN,Microsoft Testing New Game-Streaming Service,...,2018-10-08 15:20:41,297.65900,NaN,Microsoft Corp is testing a new game streaming...,Reuters,News,nyt://article/b5c2521b-0946-5436-adb1-62d71a73...,https://www.nytimes.com/reuters/2018/10/08/tec...,141,msft
1,5bbbe59b068401528a2df1ef,REUTERS,By REUTERS,[],article,NaN,NaN,Microsoft to Invest in Southeast Asian Ride-Ha...,NaN,Microsoft to Invest in Southeast Asian Ride-Ha...,...,2018-10-08 23:17:43,276.20792,NaN,Microsoft Corp is investing in Southeast Asian...,Reuters,News,nyt://article/f5290bfb-217c-5308-b763-2207b405...,https://www.nytimes.com/reuters/2018/10/08/bus...,327,msft
2,5bbb4bb5068401528a2df0a8,REUTERS,By REUTERS,[],article,NaN,NaN,Exclusive: Microsoft's $7.5 Billion GitHub Dea...,NaN,Exclusive: Microsoft's $7.5 Billion GitHub Dea...,...,2018-10-08 12:21:08,271.93472,NaN,U.S. software giant Microsoft is set to win un...,Reuters,News,nyt://article/928a254e-c6eb-5eca-896c-3556a5fa...,https://www.nytimes.com/reuters/2018/10/08/bus...,174,msft
3,5bbcabf1068401528a2df379,REUTERS,By REUTERS,[],article,NaN,NaN,Microsoft Expands Cloud Service in Push for $1...,NaN,Microsoft Expands Cloud Service in Push for $1...,...,2018-10-09 13:23:59,270.83615,NaN,Microsoft Corp said on Tuesday its expanded Az...,Reuters,News,nyt://article/94179203-ee24-5ce5-bb96-641ee251...,https://www.nytimes.com/reuters/2018/10/09/bus...,158,msft
4,5bb295c6068401528a2ddeeb,REUTERS,By REUTERS,[],article,NaN,NaN,Microsoft Co-Founder Paul Allen Treated Again ...,NaN,Microsoft Co-Founder Paul Allen Treated Again ...,...,2018-10-01 21:46:45,257.48560,NaN,Microsoft Corp co-founder Paul Allen said on M...,Reuters,News,nyt://article/2f19d5d8-3def-5d8d-9ba8-519385d2...,https://www.nytimes.com/reuters/2018/10/01/us/...,273,msft


In [7]:
msft_coding = msft_nyt[['_id', 'headline.main']].copy()
msft_coding.head()

,_id,headline.main
0,5bbb75cb068401528a2df0f5,Microsoft Testing New Game-Streaming Service
1,5bbbe59b068401528a2df1ef,Microsoft to Invest in Southeast Asian Ride-Ha...
2,5bbb4bb5068401528a2df0a8,Exclusive: Microsoft's $7.5 Billion GitHub Dea...
3,5bbcabf1068401528a2df379,Microsoft Expands Cloud Service in Push for $1...
4,5bb295c6068401528a2ddeeb,Microsoft Co-Founder Paul Allen Treated Again ...


In [8]:
for new_col in ['is_significant', 'coder', 'flag', 'notes']:
    msft_coding[new_col] = ''
msft_coding.head()

,_id,headline.main,is_significant,coder,flag,notes
0,5bbb75cb068401528a2df0f5,Microsoft Testing New Game-Streaming Service,,,,
1,5bbbe59b068401528a2df1ef,Microsoft to Invest in Southeast Asian Ride-Ha...,,,,
2,5bbb4bb5068401528a2df0a8,Exclusive: Microsoft's $7.5 Billion GitHub Dea...,,,,
3,5bbcabf1068401528a2df379,Microsoft Expands Cloud Service in Push for $1...,,,,
4,5bb295c6068401528a2ddeeb,Microsoft Co-Founder Paul Allen Treated Again ...,,,,


In [9]:
# Perhaps we think the word billion in the headline makes something
# likely to be significant.
def code_significant(text):
    if 'billion' in text.lower():
        return '1'
    else:
        return ''

In [10]:
msft_coding['is_significant'] = msft_coding['headline.main'].apply(code_significant)

In [11]:
msft_coding.head()

,_id,headline.main,is_significant,coder,flag,notes
0,5bbb75cb068401528a2df0f5,Microsoft Testing New Game-Streaming Service,,,,
1,5bbbe59b068401528a2df1ef,Microsoft to Invest in Southeast Asian Ride-Ha...,,,,
2,5bbb4bb5068401528a2df0a8,Exclusive: Microsoft's $7.5 Billion GitHub Dea...,1,,,
3,5bbcabf1068401528a2df379,Microsoft Expands Cloud Service in Push for $1...,1,,,
4,5bb295c6068401528a2ddeeb,Microsoft Co-Founder Paul Allen Treated Again ...,,,,


In this example, I write a simple function that is designed to operate on one string, and I use the `apply` method on the `headline.main` column to apply that function to each row.
Another common pattern is to merge in the values that may work, and use the coding process to check them.

# Cartesian joins

A [Cartesian product](https://en.wikipedia.org/wiki/Join_\(SQL\)#Cross_join) (also called a "cross join") is a combination of each row in one set of data with each row in another set of data.
As you can imagine, the length of the new data is $n \times m$, which gets very large as $n$ and $m$ increase.
However, when we can use small groups, these products become manageable.

## Conference call example

Imagine that we have conference call data consisting of a list of participants and then a transcript where we can isolate the names, but they don't quite match the participant list.
These are reasonably short-length lists, so we can use a Cartesian product for coding.
The idea is that we will have all of the combinations, and we expect there to be a proper match on both sides (not strictly required, but it helps), so we can produce all of the combinations and have a coder mark the proper one.

In [12]:
participants = pd.DataFrame([
    {'name': 'Abbie Executive (CEO)'},
    {'name': 'Bruce Executive (CFO)'},
    {'name': 'Charles Analyst (Firm1)'},
    {'name': 'Bella Analyst (Firm2)'}
])
participants.head()

,name
0,Abbie Executive (CEO)
1,Bruce Executive (CFO)
2,Charles Analyst (Firm1)
3,Bella Analyst (Firm2)


In [13]:
speaker = pd.DataFrame([
    {'name': 'Operator'},
    {'name': 'Abbie E.'},
    {'name': 'Bruce E.'},
    {'name': 'Charles A.'},
    {'name': 'Bella A.'}
])
speaker.head()

,name
0,Operator
1,Abbie E.
2,Bruce E.
3,Charles A.
4,Bella A.


In [14]:
call_coding = participants.merge(speaker, how='cross')
call_coding['correct'] = ''
call_coding

,name_x,name_y,correct
0,Abbie Executive (CEO),Operator,
1,Abbie Executive (CEO),Abbie E.,
2,Abbie Executive (CEO),Bruce E.,
3,Abbie Executive (CEO),Charles A.,
4,Abbie Executive (CEO),Bella A.,
5,Bruce Executive (CFO),Operator,
6,Bruce Executive (CFO),Abbie E.,
7,Bruce Executive (CFO),Bruce E.,
8,Bruce Executive (CFO),Charles A.,
9,Bruce Executive (CFO),Bella A.,


As you can see, we now have 20 rows from lists of five and four (i.e. `5 * 4`).
That is a big expansion in length, but these are fast to code.
If you look through the rows, you'll notice that you can ascertain matches very quickly, and most rows do not match.
Using this technique, we've seen coders reliably exceed 40 rows per minute.
It also has the benefit of preventing much entry, except the for `1`s entered for matches.

In [15]:
mockup = [1, 7, 13, 19]
call_coding.loc[mockup, 'correct'] = 1
call_coding

,name_x,name_y,correct
0,Abbie Executive (CEO),Operator,
1,Abbie Executive (CEO),Abbie E.,1
2,Abbie Executive (CEO),Bruce E.,
3,Abbie Executive (CEO),Charles A.,
4,Abbie Executive (CEO),Bella A.,
5,Bruce Executive (CFO),Operator,
6,Bruce Executive (CFO),Abbie E.,
7,Bruce Executive (CFO),Bruce E.,1
8,Bruce Executive (CFO),Charles A.,
9,Bruce Executive (CFO),Bella A.,


In [16]:
call_coding[call_coding['correct'] == 1]

,name_x,name_y,correct
1,Abbie Executive (CEO),Abbie E.,1
7,Bruce Executive (CFO),Bruce E.,1
13,Charles Analyst (Firm1),Charles A.,1
19,Bella Analyst (Firm2),Bella A.,1


This last form with the correct rows filtered becomes our lookup table to match these two forms of names.

# Q&A: Human coding

At the end, we will chat as one big group about human coding experiences and issues.